# Get API data
Request necessary data from https://api01.nve.no/hydrology/forecast/avalanche/v4.0.0/api

Insert relevant data into SQLite database.

General:
- date
- region
- region-ID
- danger level

Avalanche problem:
- avalanche problem name
- avalanche problem order
- trigger
- probability
- elevation

Mountain weather:
- avg. precipitation
- max precipitation (most exposed area)
- temperature (at diff. levels)
- wind speed
- wind direction